In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


# cross validation

In [2]:

# Function to create model, required for KerasClassifier
def create_model():
#     with tf.Session(config=tf.ConfigProto(
#                         intra_op_parallelism_threads=7,
#                         inter_op_parallelism_threads=7)) as sess:
#         K.set_session(sess)

    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load pima indians dataset
dataset = np.loadtxt('./data/raw-pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
%time results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Wall time: 1min 51s
0.692652086761


In [ ]:
import tensorflow as tf
from keras.backend import tensorflow_backend as K

def create_model():
    # use more cpu
    with tf.Session(config=tf.ConfigProto(
                        intra_op_parallelism_threads=7,
                        inter_op_parallelism_threads=7)) as sess:
        K.set_session(sess)

        # create model
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load pima indians dataset
dataset = np.loadtxt('./data/raw-pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
%time results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Wall time: 1min 35s
0.686090226709


# grid search CV

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:
import tensorflow as tf
from keras.backend import tensorflow_backend as K

# Function to create model, required for KerasClassifier
def create_model():
    # use more cpu
    with tf.Session(config=tf.ConfigProto(
                        intra_op_parallelism_threads=7,
                        inter_op_parallelism_threads=7)) as sess:
        K.set_session(sess)
        
        # create model
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load pima indians dataset
dataset = np.loadtxt('./data/raw-pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

%time grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
%time grid_result = grid.fit(X, Y)
# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Wall time: 0 ns


### Minimial example of gridsearchcv

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=5)
%time clf.fit(iris.data, iris.target)